In [54]:
class Card:
    suits = ["Clubs", "Diamonds", "Hearts", "Spades"]
    ranks = ["narf", "Ace", "2", "3", "4", "5", "6", "7", "8", "9", "10", "Jack", "Queen", "King"]
    
    def __init__(self, suit=0, rank=0):
        self.suit = suit
        self.rank = rank
        
    def __str__(self):
        return (self.ranks[self.rank] + " of " + self.suits[self.suit])
    
    def cmp(self, other):
        # Check the suits
        if self.suit > other.suit: return 1
        if self.suit < other.suit: return -1
        # Suits are the same, check ranks
        if self.rank > other.rank: return 1
        if self.rank < other.rank: return -1
        # Ranks are the same, it's a tie
        return 0
    
    # six special methods that do the overloading of each of the relational operators for us
    def __eq__(self, other):
        return self.cmp(other) == 0
    def __le__(self, other):
        return self.cmp(other) <= 0
    def __ge__(self, other):
        return self.cmp(other) >= 0
    def __gt__(self, other):
        return self.cmp(other) > 0
    def __lt__(self, other):
        return self.cmp(other) < 0
    def __ne__(self, other):
        return self.cmp(other) != 0
    
# The initialization method creates the attribute cards and generates the standard pack of fifty-two cards
class Deck:
    def __init__(self):
        self.cards = []
        for suit in range(4):
            for rank in range(1, 14):
                self.cards.append(Card(suit, rank))
                
    # Printing the deck
    def __str__(self):
        s = ""
        for i in range(len(self.cards)):
            s = s + " " * i + str(self.cards[i]) + "\n"
        return s
    
    def shuffle(self):
        import random
        rng = random.Random() # Create a random generator
        rng.shuffle(self.cards) # Use generator's shuffle method

    # Removing and dealing cards
    def remove(self, card):
        if card in self.cards:
            self.cards.remove(card)
            return True
        else:
            return False
        
    # Remove and return the top card.
    def pop(self):
        return self.cards.pop()
    
    # returns True if the deck contains no cards
    def is_empty(self):
        return self.cards == []
    
    def deal(self, hands, num_cards=999):
        num_hands = len(hands)
        for i in range(num_cards):
            if self.is_empty():
                break # break if out of cards
            card =  self.pop() # take the top card
            hand = hands[i % num_hands] # whose turn is next?
            hand.add(card) # add the card to the hand
    
    
class Hand(Deck):
    def __init__(self, name=""):
        self.cards = []
        self.name = name
        
    # remove is already added in Deck class. let's write add function
    def add(self, card):
        self.cards.append(card)
        
    def __str__(self):
        s = "Hand " + self.name
        if self.is_empty():
            s += " is empty\n"
        else:
            s += " contains\n"
        return s + Deck.__str__(self)
    
    
class CardGame:
    def __init__(self):
        self.deck = Deck()
        self.deck.shuffle()
        
class OldMaidHand(Hand):
    def remove_matches(self):
        count = 0
        original_cards = self.cards[:]
        for card in original_cards:
            match = Card(3 - card.suit, card.rank)
            if match in self.cards:
                self.cards.remove(card)
                self.cards.remove(match)
                print("Hand {0}: {1} matches {2}".format(self.name, card, match))
                count += 1
        return count
    
    
class OldMaidGame(CardGame):
    def play(self, names):
        # Remove queen of clubs
        self.deck.remove(Card(0,12))
        
        # Make a hand for each player
        self.hands = []
        for name in names:
            self.hands.append(OldMaidHand(name))
            
        # Deal the cards
        self.deck.deal(self.hands)
        print("------------ Cards have been dealt")
        self.print_hands()
        
        # Remove initial matches
        matches = self.remove_all_matches()
        print("------------ Matches discarded, play begins")
        self.print_hands()
        
        # Play until all 50 cards are mathed
        turn  = 0
        num_hands = len(self.hands)
        while matches < 25:
            matches += self.play_one_turn(turn)
            turn = (turn + 1) % num_hands
            
        print("------------ Game is over!")
        self.print_hands()
        
    def remove_all_matches(self):
        count = 0
        for hand in self.hands:
            count += hand.remove_matches()
        return count
    
    def play_one_turn(self, i):
        if self.hands[i].is_empty():
            return 0
        neighbor = self.find_neighbor(i)
        picked_card = self.hands[neighbor].pop()
        self.hands[i].add(picked_card)
        print("Hand", self.hands[i].name, "picked", picked_card)
        count = self.hands[i].remove_matches()
        self.hands[i].shuffle()
        return count
    
    def find_neighbor(self, i):
        num_hands = len(self.hands)
        for next in range(1, num_hands):
            neighbor = (i + next) % num_hands
            if not self.hands[neighbor].is_empty():
                return neighbor
            
    def print_hands(self):
        for hand in self.hands:
            print(hand)

In [55]:
deck = Deck()
deck.shuffle()
hand = Hand("frank")
deck.deal([hand], 5)
print(hand)

Hand frank contains
4 of Clubs
 3 of Hearts
  3 of Diamonds
   Queen of Hearts
    Jack of Spades



In [56]:
game = CardGame()
hand = OldMaidHand("frank")
game.deck.deal([hand], 13)
print(hand)

Hand frank contains
5 of Spades
 King of Spades
  2 of Spades
   4 of Clubs
    5 of Clubs
     Ace of Spades
      King of Clubs
       2 of Hearts
        10 of Diamonds
         9 of Spades
          5 of Hearts
           6 of Spades
            Ace of Hearts



In [57]:
hand.remove_matches()

Hand frank: 5 of Spades matches 5 of Clubs
Hand frank: King of Spades matches King of Clubs


2

In [58]:
print(hand)

Hand frank contains
2 of Spades
 4 of Clubs
  Ace of Spades
   2 of Hearts
    10 of Diamonds
     9 of Spades
      5 of Hearts
       6 of Spades
        Ace of Hearts



In [64]:
# import  cards
# game = cards.OldMaidGame()
game = OldMaidGame()
game.play(["Allen", "Jeff", "Chris"])

------------ Cards have been dealt
Hand Allen contains
10 of Spades
 3 of Clubs
  8 of Diamonds
   3 of Hearts
    7 of Spades
     4 of Diamonds
      5 of Clubs
       10 of Hearts
        Queen of Spades
         Ace of Diamonds
          3 of Diamonds
           2 of Spades
            Jack of Diamonds
             9 of Spades
              8 of Spades
               9 of Clubs
                King of Hearts

Hand Jeff contains
4 of Spades
 7 of Diamonds
  6 of Diamonds
   8 of Hearts
    6 of Hearts
     10 of Diamonds
      Jack of Spades
       Queen of Diamonds
        Ace of Clubs
         3 of Spades
          Jack of Clubs
           2 of Clubs
            King of Clubs
             7 of Hearts
              5 of Hearts
               King of Spades
                10 of Clubs

Hand Chris contains
7 of Clubs
 5 of Spades
  Ace of Spades
   6 of Spades
    2 of Diamonds
     4 of Hearts
      5 of Diamonds
       Ace of Hearts
        Jack of Hearts
         Queen of Hearts
 